In [ ]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz

from fiftyone.utils.huggingface import load_from_hub

In [ ]:
# Load dataset from Hugging Face Hub
# https://huggingface.co/datasets/Voxel51/fisheye8k
dataset = load_from_hub("Voxel51/fisheye8k", name ="fisheye8k-100", max_samples=100, overwrite=True)

In [ ]:
# Iterate over samples
for sample in dataset:
    print(sample)
    break

In [ ]:
# Compute and visualize image embeddings
model_embeddings = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
fob.compute_visualization(
    dataset,
    model=model_embeddings,
    method="tsne",
    brain_key="mobilenet_tsne",
    num_workers=16
)

In [ ]:
# Zero-Shot Object Detection based on existing object classes
# Models from Hugging Face: https://huggingface.co/models?pipeline_tag=zero-shot-object-detection&library=transformers&sort=trending
dataset_classes = dataset.default_classes
print(f"Object classes in dataset: {dataset_classes}")

# Grounding DINO
model_zero_shot_grounding_dino = foz.load_zoo_model(
    "zero-shot-detection-transformer-torch",
    name_or_path="IDEA-Research/grounding-dino-base",
    classes=dataset_classes,
)
dataset.apply_model(model_zero_shot_grounding_dino, label_field="grounding_dino", confidence_thresh=0.2, progress=True)

# OwlV2
model_zero_shot_owlv2 = foz.load_zoo_model(
    "zero-shot-detection-transformer-torch",
    name_or_path="google/owlv2-base-patch16-ensemble",
    classes=dataset_classes,
)
dataset.apply_model(model_zero_shot_owlv2, label_field="owlv2", confidence_thresh=0.2, progress=True)

In [ ]:
# Evaluate detections
dataset.evaluate_detections(
    "grounding_dino",
    gt_field="detections",
    eval_key="eval_grounding_dino",
    compute_mAP=True,
)

dataset.evaluate_detections(
    "owlv2",
    gt_field="detections",
    eval_key="eval_owlv2",
    compute_mAP=True,
)

In [ ]:
# Launch Voxel51 GUI
session = fo.launch_app(dataset)